In [29]:
import pandas as pd
import numpy as np
import re

Exclusion criteria:

[auditc_total] < 4 or 

(([screen_sleep_trouble]<2 AND [jenkins_toggle]=0) OR ([screen_sleep_trouble]='' AND [jenkins_toggle]<1)) or

[age_v2] <> '1' or [webscreen_smartphone]=2 or [webscreen_smartphone_have]=0

In [18]:
screen = pd.read_csv('webscreen.csv')
# exclude beta
screen = screen[screen['record_id']>=777]
# exclude duplicates
screen = screen[screen['webscreen_status']!=5]
screen.head()

,record_id,ts,age_v2,screen_drinking_problem,screen_drink_frequency,screen_drink_amount,screen_drink_five,auditc_total,screen_sleep_trouble,webscreen_smartphone_have,...,webscreen_exclusionreason___0,webscreen_exclusionreason___1,webscreen_exclusionreason___2,webscreen_exclusionreason___3,webscreen_exclusionreason___4,webscreen_exclusionreason___5,webscreen_exclusionreason___6,webscreen_exclusionreason___20,webscreen_exclusionreason___99,tracking_form_complete
675,777,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
676,778,2021-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
677,779,2021-09-12,NaN,0.0,4.0,0.0,0.0,4.0,NaN,NaN,...,0,1,0,0,0,1,0,0,0,2
678,780,2021-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
679,781,2021-09-12,NaN,1.0,4.0,1.0,2.0,7.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [20]:
len(screen)

4101

In [21]:
# total phone screens
len(screen.loc[screen['webscreen_status']==9])

322

In [22]:
# number that passed phone screen
len(screen.loc[(screen['webscreen_status']==9) & (screen.filter(regex=('.*webscreen_exclusionreason')).sum(axis=1)==0)])

287

In [23]:
# number that failed phone screen
len(screen.loc[(screen['webscreen_status']==9) & (screen.filter(regex=('.*webscreen_exclusionreason')).sum(axis=1)>0)])

35

In [17]:
#screen.filter(regex=('.*webscreen_exclusionreason')).loc[screen['webscreen_status']==9].sum(axis=0)

In [194]:
failed_webscreen = screen[(screen['webscreen_status'].isna()) | (screen['webscreen_status']!=9)]
#len(failed_webscreen)
failed_webscreen['fail_aud'] = np.nan
failed_webscreen['fail_sleep'] = np.nan
failed_webscreen['fail_age'] = np.nan
failed_webscreen['fail_no_phone'] = np.nan
failed_webscreen['fail_android'] = np.nan
failed_webscreen['fail_not_complete'] = np.nan
failed_webscreen['fail_lost_contact'] = np.nan

/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_33405/1775618466.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failed_webscreen['fail_aud'] = np.nan
/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_33405/1775618466.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failed_webscreen['fail_sleep'] = np.nan
/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_33405/1775618466.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

Exclusion criteria:

[auditc_total] < 4 or 

(([screen_sleep_trouble]<2 AND [jenkins_toggle]=0) OR ([screen_sleep_trouble]='' AND [jenkins_toggle]<1)) or

[age_v2] <> '1' or [webscreen_smartphone]=2 or [webscreen_smartphone_have]=0

In [195]:
for i, row in failed_webscreen.iterrows():
    other_reason = False
    if row['auditc_total'] < 4:
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_aud'] = 'AUD'
        other_reason = True
    if (row['screen_sleep_trouble']<2 and row['jenkins_toggle']==0) or (not np.isfinite(row['screen_sleep_trouble']) and row['jenkins_toggle']<1):
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_sleep'] = 'SLEEP'
        other_reason = True
    if row['age_v2'] != 1:
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_age'] = 'AGE'
        other_reason = True
    if row['webscreen_smartphone_have'] == 0 or not np.isfinite(row['webscreen_smartphone_have']):
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_no_phone'] = 'PHONE'
        other_reason = True
    if row['webscreen_smartphone'] == 2:
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_android'] = 'ANDROID'
        other_reason = True
    if row['screen_form_complete'] == 0 and not other_reason:
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_not_complete'] = 'FORM_NOT_COMPLETE'
    if row['screen_form_complete'] != 0 and not other_reason:
        failed_webscreen.loc[failed_webscreen['record_id']==row['record_id'], 'fail_lost_contact'] = 'NOT_INTERESTED/LOST_CONTACT'


In [125]:
#failed_webscreen.to_csv('test.csv', index=False)

In [196]:
failed_webscreen['exclusion_reasons'] = failed_webscreen[['fail_aud','fail_sleep','fail_age','fail_no_phone','fail_android','fail_not_complete','fail_lost_contact']].fillna('').agg(','.join, axis=1)
failed_webscreen['exclusion_reasons'] = failed_webscreen['exclusion_reasons'].apply(lambda x: x.strip(','))

/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_33405/1196068046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failed_webscreen['exclusion_reasons'] = failed_webscreen[['fail_aud','fail_sleep','fail_age','fail_no_phone','fail_android','fail_not_complete','fail_lost_contact']].fillna('').agg(','.join, axis=1)
/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_33405/1196068046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failed_webscreen['exclusion_reasons'] = failed_webscreen[

In [197]:
failed_webscreen.loc[failed_webscreen['exclusion_reasons']==''].to_csv('test1.csv', index=False)

In [198]:
failed_webscreen['exclusion_reasons'].value_counts()

AGE,PHONE                      1365
SLEEP,AGE,PHONE                 716
ANDROID                         461
PHONE                           433
NOT_INTERESTED/LOST_CONTACT     274
FORM_NOT_COMPLETE               205
AUD,,,PHONE                      88
AUD                              59
SLEEP,,PHONE                     43
AUD,,,,ANDROID                   41
AUD,,AGE,PHONE                   31
AUD,SLEEP,AGE,PHONE              27
AUD,SLEEP,,PHONE                 17
SLEEP                             5
AUD,SLEEP                         4
SLEEP,,,ANDROID                   4
AUD,SLEEP,,,ANDROID               2
AGE,,ANDROID                      2
AUD,,AGE                          1
AGE                               1
Name: exclusion_reasons, dtype: int64

In [199]:
failed_webscreen['exclusion_reasons'].value_counts().to_csv('exclusion_reasons.csv', index=True)